In [ ]:
import pandas as pd
import numpy as np 

import matplotlib.pyplot as plt 
import seaborn as sns


In [ ]:
weatherAUS = pd.read_csv('../input/weather-dataset-rattle-package/weatherAUS.csv')
weatherAUS.head()

# Let's Explore the Data

In [ ]:
#length of the dataset 
print('dataset have {} rows and {} columns'.format(weatherAUS.shape[0],weatherAUS.shape[1]))

In [ ]:
print('Name of columns:')
for i in weatherAUS.columns:
    
    print(i)

In [ ]:
#viewing summary of dataset:

weatherAUS.info()

from above we can clearly see that data contains both numerical as well as categorical variables.
viewing it's statistical properties of dataset help to understand behaviour of the features.

In [ ]:
#for all numerical variables 
weatherAUS.describe()



In [ ]:
#for all categorical 
weatherAUS.describe(include = ['object'])

It's always better to look numerical and categorical variable separately 

Now,let's look into the number of null values in each features.

In [ ]:
weatherAUS.isnull().sum()

droping rows where target columns (RainTomorrow) are null. 

In [ ]:
weatherAUS['RainTomorrow'].unique()

In [ ]:
#droping rows when there is null values in target columnn..

weatherAUS = weatherAUS[~weatherAUS['RainTomorrow'].isnull()]

In [ ]:
#let's Start our Eda with out target variable

weatherAUS['RainTomorrow'].value_counts()/len(weatherAUS)

As we can see that our target columns consist of  77.58% No and 22.42% Yes

In [ ]:
fig , ax = plt.subplots(figsize = (6,5))
ax = sns.countplot(y= 'RainTomorrow', data=weatherAUS )

In [ ]:
#let's save categorical variables and numerical variables name in a list

categorical = [var for var in weatherAUS.columns if weatherAUS[var].dtype =='O']

Numerical = [var for var  in weatherAUS.columns if weatherAUS[var].dtype !='O' ]


print('There are {} categorical variables\nThere names are: {}\n'.format(len(categorical),categorical))

print('There are {} numerical variables\nThere names are: {}'.format(len(Numerical),Numerical))

In [ ]:
100*weatherAUS[categorical].isna().sum()/len(weatherAUS[categorical])

In [ ]:
#Let's explore within categorical variable

cat_variables_withnull = [var for var in categorical if weatherAUS[var].isna().sum() !=0]

weatherAUS[cat_variables_withnull].isna().sum()



In [ ]:
#replacing null values of categorical variables with None.
for cat in cat_variables_withnull:
    
    weatherAUS.loc[:,cat] =  weatherAUS.loc[:,cat].fillna('None')

In [ ]:
#now our categorical variables have no null values but if we here date column is coming inside categorical column which is wrong 
#so let's change the date column data type 

print(weatherAUS['Date'].dtypes)

weatherAUS['Date'] = pd.to_datetime(weatherAUS['Date'])

print(weatherAUS['Date'].dtypes)


categorical = [var for var in weatherAUS.columns if weatherAUS[var].dtype =='O']


print('There are {} categorical variables\nThere names are: {}\n'.format(len(categorical),categorical))

In [ ]:
#finding numerical variables which consist of Null values...
num_variables_withnull = [var for var in Numerical if weatherAUS[var].isna().sum() !=0]

weatherAUS[num_variables_withnull].isna().sum()

In [ ]:
#finding the % of null values : 

100*weatherAUS[Numerical].isna().sum()/len(weatherAUS[Numerical])

In [ ]:
#checking for correlation between the variables:
correlation = weatherAUS.corr()
plt.figure(figsize=(16,12))
plt.title('Correlation Heatmap of Rain in Australia Dataset')
ax = sns.heatmap(correlation, square=True, annot=True, fmt='.2f', linecolor='white')
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
ax.set_yticklabels(ax.get_yticklabels(), rotation=30)           
plt.show()


From above matrix we can see that following infernces:
    
    MaxTemp,MinTemp,Temp9am,Temp3pm are highly correlated.
    
    Sunshine, Cloud9am, Cloud3am are also highly correlated.
    

In [ ]:
sns.pairplot(weatherAUS[Numerical], kind = 'scatter', diag_kind = 'hist', palette = 'Rainbow')
plt.show()

In [ ]:
#creating Year, Month, Week from Date column...
weatherAUS.loc[:,'year']  = weatherAUS.loc[:,'Date'].dt.year
weatherAUS.loc[:,'month']  = weatherAUS.loc[:,'Date'].dt.month
weatherAUS.loc[:,'week']  = weatherAUS.loc[:,'Date'].dt.week

In [ ]:
#creating time based CV split 


fold_split =  {
    0 : {'train': [2008] , 'test' : [2009]},
    1 : {'train': [2008,2009] , 'test' : [2010]},
    2 : {'train': [2008,2009,2010] , 'test' : [2011]},
    3 : {'train': [2008,2009,2010,2011] , 'test' : [2012]},
    4 : {'train': [2008,2009,2010,2011,2012], 'test' : [2013]},
    5 : {'train': [2008,2009,2010,2011,2012,2013], 'test' : [2014]},
    6 : {'train': [2008,2009,2010,2011,2012,2013,2014], 'test' : [2015]},
    7 : {'train': [2008,2009,2010,2011,2012,2013,2014,2015], 'test' : [2016]}
            
}

In [ ]:
#filling null values for numerical variables: 
for i in num_variables_withnull:
    if weatherAUS[i].isna().sum() >0:
        weatherAUS.loc[:,i] = weatherAUS.groupby(
     ['Location',
     'WindGustDir',
     'WindDir9am',
     'WindDir3pm',
     'year',
     'month',
     'week']
    )[i].transform(
        lambda grp: grp.fillna(np.mean(grp))
    )
        if weatherAUS[i].isna().sum() >0:
            weatherAUS.loc[:,i] = weatherAUS.groupby(
     ['Location',
     'WindGustDir',
     'WindDir9am',
     'year',
     'month',
     'week']
    )[i].transform(
        lambda grp: grp.fillna(np.mean(grp))
    )
            if weatherAUS[i].isna().sum() >0:
                weatherAUS.loc[:,i] = weatherAUS.groupby(
     ['Location',
     'WindGustDir',
     'year',
     'month',
     'week']
    )[i].transform(
        lambda grp: grp.fillna(np.mean(grp))
    )
                if weatherAUS[i].isna().sum() >0:
                    weatherAUS.loc[:,i] = weatherAUS.groupby(
     ['Location',
     'year',
     'month',
     'week']
    )[i].transform(
        lambda grp: grp.fillna(np.mean(grp))
    )        
                    if weatherAUS[i].isna().sum() >0:
                        weatherAUS.loc[:,i] = weatherAUS.groupby(
     [
     'year',
     'month',
     'week']
    )[i].transform(
        lambda grp: grp.fillna(np.mean(grp))
    )





In [ ]:
#Creating label encoding for categorical variables:
from sklearn import preprocessing
categorical =['Location',
 'WindGustDir',
 'WindDir9am',
 'WindDir3pm',
 'RainToday','RainTomorrow']
for feat in categorical:
    lbl_enc = preprocessing.LabelEncoder()
    weatherAUS.loc[:,feat] = lbl_enc.fit_transform(weatherAUS.loc[:,feat])

In [ ]:
#just trying to fit Randomforest model on Different folds and printing it's Roc
for i in list(fold_split.keys()):
    features = [ 'Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation',
           'Sunshine', 'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm',
           'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
           'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
           'Temp3pm', 'RainToday', 'year', 'month', 'week']
    train = weatherAUS[weatherAUS['year'].isin(fold_split[i]['train'])]
    test = weatherAUS[weatherAUS['year'].isin(fold_split[i]['test'])]
    X_train = train[features]
    y_train = train['RainTomorrow']
    X_test = test[features]
    y_test = test['RainTomorrow']
    from sklearn.ensemble import RandomForestClassifier
    model = RandomForestClassifier()
    model.fit(X_train,y_train)
    preds=model.predict_proba(X_test)[:,1]
    from sklearn.metrics import accuracy_score , confusion_matrix,roc_auc_score
    print('Roc value for fold {}: {}'.format(i,roc_auc_score(y_test,preds)))

In [ ]:
#predicting probabilties :) 
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()

model.fit(X_train,y_train)
preds=model.predict_proba(X_test)[:,1]
from sklearn.metrics import accuracy_score , confusion_matrix,roc_auc_score, roc_curve
roc_auc_score(y_test,preds)

In [ ]:
#checking for confusion metrics
confusion_matrix(y_test,model.predict(X_test))

In [ ]:
a,b,c =roc_curve(y_test,preds)
accuracy_ls = []

for thres in c:
    y_pred = np.where(preds>thres,1,0)
    accuracy_ls.append(accuracy_score(y_test,y_pred,normalize=True))

accuracy_df = pd.concat([pd.Series(c),pd.Series(accuracy_ls)],axis = 1 )  
accuracy_df.columns = ['threshold','accuracy']
accuracy_df[accuracy_df['accuracy'] == accuracy_df['accuracy'].max()]

In [ ]:
#creating json for finding hpyerparameters :) 
param_grid = {
    "n_estimators": np.arange(100,1500,100),
    "max_depth" : np.arange(1,31),
    "criterion" : ['gini','entropy']
}
from sklearn import model_selection

In [ ]:
g = model_selection.RandomizedSearchCV(estimator=model,param_distributions=param_grid,scoring='accuracy',verbose =10,n_jobs =1,cv=5)

In [ ]:
g.fit(X_train,y_train)

In [ ]:
#finding the best accuracy score from it :) 
g.best_score_

In [ ]:
#we will get the parameters and try to fit the model again:
g.best_estimator_.get_params()

In [ ]:
#random forest with optimal parameters 
model = RandomForestClassifier(max_depth=25,n_estimators=1400,criterion='entropy')
model.fit(X_train,y_train)
preds=model.predict_proba(X_test)[:,1]
roc_auc_score(y_test,preds)

In [ ]:
#finding the optimal threshold for accuracy
accuracy_ls = []

for thres in c:
    y_pred = np.where(preds>thres,1,0)
    accuracy_ls.append(accuracy_score(y_test,y_pred,normalize=True))

accuracy_df = pd.concat([pd.Series(c),pd.Series(accuracy_ls)],axis = 1 )  
accuracy_df.columns = ['threshold','accuracy']
accuracy_df[accuracy_df['accuracy'] == accuracy_df['accuracy'].max()]    

In [ ]:
confusion_matrix(y_test,np.where(preds>0.48,1,0))

In [ ]:
import xgboost as xgb
clf = xgb.XGBClassifier()

param_grid = {
        'silent': [False],
        'max_depth': [6, 10, 15, 20],
        'learning_rate': [0.001, 0.01, 0.1, 0.2, 0,3],
        'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bytree': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bylevel': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'min_child_weight': [0.5, 1.0, 3.0, 5.0, 7.0, 10.0],
        'gamma': [0, 0.25, 0.5, 1.0],
        'reg_lambda': [0.1, 1.0, 5.0, 10.0, 50.0, 100.0],
        'n_estimators': [100]}

'''fit_params = {'eval_metric': 'mlogloss',
              'early_stopping_rounds': 10,
              'eval_set': [(X_test, y_test)]}'''

rs_clf = model_selection.RandomizedSearchCV(clf, param_grid, n_iter=20,
                            n_jobs=1, verbose=2, cv=2,
                            scoring='accuracy', refit=False, random_state=42)
print("Randomized search..")
#search_time_start = time.time()
rs_clf.fit(X_train, y_train)
#print("Randomized search time:", time.time() - search_time_start)

best_score = rs_clf.best_score_
best_params = rs_clf.best_params_
print("Best score: {}".format(best_score))
print("Best params: ")
for param_name in sorted(best_params.keys()):
    print('%s: %r' % (param_name, best_params[param_name]))